# ベクトル埋め込み、OpenAI、CQL を通じた Cassandra / Astra DB を使った哲学

### CassIO バージョン

このクイックスタートでは、OpenAIのベクトル埋め込みと[Apache Cassandra®](https://cassandra.apache.org)、または同等のDataStax [Astra DB through CQL](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html)をベクトルストアとしてデータ永続化に使用して、「哲学の名言検索・生成器」を構築する方法を学びます。

このノートブックの基本的なワークフローは以下に概説されています。有名な哲学者による多数の名言のベクトル埋め込みを評価・保存し、それらを使用して強力な検索エンジンを構築し、その後、新しい名言の生成器まで作成します！

このノートブックは、ベクトル検索の標準的な使用パターンの一部を例示しており、[Cassandra](https://cassandra.apache.org/doc/trunk/cassandra/vector-search/overview.html) / [Astra DB through CQL](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html)のベクトル機能を使い始めることがいかに簡単かを示しています。

ベクトル検索とテキスト埋め込みを使用して質問応答システムを構築する背景については、この優れた実践的なノートブックをご確認ください：[Question answering using embeddings](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb)。

#### _フレームワークを選択_

このノートブックは[CassIO library](https://cassio.org)を使用していますが、同じタスクを達成するための他の技術選択肢もカバーしています。他のオプションについては、このフォルダの[README](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases/cassandra_astradb)をご確認ください。このノートブックはColabノートブックまたは通常のJupyterノートブックとして実行できます。

目次：
- セットアップ
- DB接続の取得
- OpenAIへの接続
- ベクトルストアへの名言の読み込み
- 使用例1：**名言検索エンジン**
- 使用例2：**名言生成器**
- （オプション）ベクトルストアでのパーティショニングの活用

### 仕組み

**インデックス化**

各引用文はOpenAIの`Embedding`を使用してエンベディングベクトルに変換されます。これらは後の検索で使用するためにVector Storeに保存されます。著者名やその他のいくつかの事前計算されたタグを含むメタデータも一緒に保存され、検索のカスタマイズを可能にします。

![1_vector_indexing](https://user-images.githubusercontent.com/14221764/282440878-dc3ed680-7d0e-4b30-9a74-d2d66a7394f7.png)

**検索**

提供された検索用引用文に類似した引用文を見つけるために、後者はその場でエンベディングベクトルに変換され、このベクトルを使用してストアから類似したベクトル、つまり以前にインデックス化された類似の引用文を検索します。検索は追加のメタデータによってオプションで制約することができます（「スピノザによるこれに似た引用文を見つけて...」など）。

![2_vector_search](https://user-images.githubusercontent.com/14221764/282440908-683e3ee1-0bf1-46b3-8621-86c31fc7f9c9.png)

ここでの重要なポイントは、「内容が類似した引用文」がベクトル空間では、互いに距離的に近いベクトルに変換されることです。つまり、ベクトル類似性検索は効果的に意味的類似性を実装します。_これがベクトルエンベディングが非常に強力である主な理由です。_

以下のスケッチはこのアイデアを伝えようとしています。各引用文は、ベクトルに変換されると空間内の点になります。この場合、OpenAIのエンベディングベクトルは他の多くのベクトルと同様に_単位長_に正規化されているため、球面上の点になります。そして、この球面は実際には3次元ではなく、1536次元です！

つまり、本質的にベクトル空間での類似性検索は、クエリベクトルに最も近いベクトルを返します：

![3_vector_space](https://user-images.githubusercontent.com/14221764/262321363-c8c625c1-8be9-450e-8c68-b1ed518f990d.png)

**生成**

提案（トピックまたは仮の引用文）が与えられると、検索ステップが実行され、最初に返された結果（引用文）がLLMプロンプトに入力されます。このプロンプトは生成モデルに対して、渡された例_および_初期提案に沿った新しいテキストを発明するよう求めます。

![4_quote_generation](https://user-images.githubusercontent.com/14221764/282440927-d56f36eb-d611-4342-8026-7736edc6f5c9.png)

## セットアップ

まず、必要なパッケージをインストールしてください：

In [1]:
!pip install --quiet "cassio>=0.1.3" "openai>=1.0.0" datasets

In [2]:
from getpass import getpass
from collections import Counter

import cassio
from cassio.table import MetadataVectorCassandraTable

import openai
from datasets import load_dataset

## DB接続を取得する

CQLを通じてAstra DBに接続するには、以下の2つが必要です：
- "Database Administrator"ロールを持つToken（`AstraCS:...`のような形式）
- データベースID（`3df2a5b6-...`のような形式）

    両方の文字列を確実に取得してください。これらは、サインイン後に[Astra UI](https://astra.datastax.com)で取得できます。詳細については、こちらを参照してください：[データベースID](https://awesome-astra.github.io/docs/pages/astra/faq/#where-should-i-find-a-database-identifier)と[Token](https://awesome-astra.github.io/docs/pages/astra/create-token/#c-procedure)。

_Cassandraクラスターに接続したい_場合（ただし、Vector Searchを[サポート](https://cassandra.apache.org/doc/trunk/cassandra/vector-search/overview.html)している必要があります）、`cassio.init(session=..., keyspace=...)`を適切なSessionとクラスター用のキースペース名に置き換えてください。

In [3]:
astra_token = getpass("Please enter your Astra token ('AstraCS:...')")
database_id = input("Please enter your database id ('3df2a5b6-...')")

Please enter your Astra token ('AstraCS:...') ········
Please enter your database id ('3df2a5b6-...') 01234567-89ab-dcef-0123-456789abcdef


In [4]:
cassio.init(token=astra_token, database_id=database_id)

### DB接続の作成

以下は、CQLを通じてAstra DBへの接続を作成する方法です：

_（ちなみに、以下の`Cluster`インスタンス化の[パラメータを変更する](https://docs.datastax.com/en/developer/python-driver/latest/getting_started/#connecting-to-cassandra)ことで、任意のCassandraクラスター（ベクター機能を提供する限り）も使用できます。）_

### CassIOを通じたベクトルストアの作成

ベクトルをサポートし、メタデータを備えたテーブルが必要です。これを「philosophers_cassio」と呼びます：

In [5]:
v_table = MetadataVectorCassandraTable(table="philosophers_cassio", vector_dimension=1536)

## OpenAIに接続

### シークレットキーの設定

In [6]:
OPENAI_API_KEY = getpass("Please enter your OpenAI API Key: ")

Please enter your OpenAI API Key:  ········


### 埋め込みのテストコール

入力テキストのリストに対して埋め込みベクトルを取得する方法を素早く確認してください：

In [7]:
client = openai.OpenAI(api_key=OPENAI_API_KEY)
embedding_model_name = "text-embedding-3-small"

result = client.embeddings.create(
    input=[
        "This is a sentence",
        "A second sentence"
    ],
    model=embedding_model_name,
)

_注意: 上記はOpenAI v1.0+の構文です。以前のバージョンを使用している場合、埋め込みを取得するコードは異なって見えます。_

In [8]:
print(f"len(result.data)              = {len(result.data)}")
print(f"result.data[1].embedding      = {str(result.data[1].embedding)[:55]}...")
print(f"len(result.data[1].embedding) = {len(result.data[1].embedding)}")

len(result.data)              = 2
result.data[1].embedding      = [-0.010821706615388393, 0.001387271680869162, 0.0035479...
len(result.data[1].embedding) = 1536


## ベクトルストアに引用文を読み込む

_注意: 上記はOpenAI v1.0+の構文です。以前のバージョンを使用している場合、埋め込みを取得するコードは異なって見えます。_

In [9]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]

簡単な検査：

In [10]:
print("An example entry:")
print(philo_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


データセットのサイズを確認してください：

In [11]:
author_count = Counter(entry["author"] for entry in philo_dataset)
print(f"Total: {len(philo_dataset)} quotes. By author:")
for author, count in author_count.most_common():
    print(f"    {author:<20}: {count} quotes")

Total: 450 quotes. By author:
    aristotle           : 50 quotes
    schopenhauer        : 50 quotes
    spinoza             : 50 quotes
    hegel               : 50 quotes
    freud               : 50 quotes
    nietzsche           : 50 quotes
    sartre              : 50 quotes
    plato               : 50 quotes
    kant                : 50 quotes


### ベクターストアに引用文を挿入する

引用文の埋め込みを計算し、テキスト自体と後で使用予定のメタデータと共にベクターストアに保存します。著者は、引用文自体に既に含まれている「tags」と共にメタデータフィールドとして追加されることに注意してください。

速度を最適化し、呼び出し回数を削減するため、OpenAIの埋め込みサービスに対してバッチ呼び出しを実行します。

_（注意：より高速な実行のため、CassandraとCassIOでは並行挿入が可能ですが、よりわかりやすいデモコードにするため、ここでは使用していません。）_

In [12]:
BATCH_SIZE = 50

num_batches = ((len(philo_dataset) + BATCH_SIZE - 1) // BATCH_SIZE)

quotes_list = philo_dataset["quote"]
authors_list = philo_dataset["author"]
tags_list = philo_dataset["tags"]

print("Starting to store entries:")
for batch_i in range(num_batches):
    b_start = batch_i * BATCH_SIZE
    b_end = (batch_i + 1) * BATCH_SIZE
    # compute the embedding vectors for this batch
    b_emb_results = client.embeddings.create(
        input=quotes_list[b_start : b_end],
        model=embedding_model_name,
    )
    # prepare the rows for insertion
    print("B ", end="")
    for entry_idx, emb_result in zip(range(b_start, b_end), b_emb_results.data):
        if tags_list[entry_idx]:
            tags = {
                tag
                for tag in tags_list[entry_idx].split(";")
            }
        else:
            tags = set()
        author = authors_list[entry_idx]
        quote = quotes_list[entry_idx]
        v_table.put(
            row_id=f"q_{author}_{entry_idx}",
            body_blob=quote,
            vector=emb_result.embedding,
            metadata={**{tag: True for tag in tags}, **{"author": author}},
        )
        print("*", end="")
    print(f" done ({len(b_emb_results.data)})")

print("\nFinished storing entries.")

Starting to store entries:
B ************************************************** done (50)
B ************************************************** done (50)
B ************************************************** done (50)
B ************************************************** done (50)
B ************************************************** done (50)
B ************************************************** done (50)
B ************************************************** done (50)
B ************************************************** done (50)
B ************************************************** done (50)

Finished storing entries.


## ユースケース1: **引用検索エンジン**

quote-search機能については、まず入力されたquoteをベクトルに変換し、それを使用してストアにクエリを実行する必要があります（検索呼び出しにオプションのメタデータを処理することに加えて）。

再利用しやすくするために、検索エンジンの機能を関数にカプセル化します：

In [13]:
def find_quote_and_author(query_quote, n, author=None, tags=None):
    query_vector = client.embeddings.create(
        input=[query_quote],
        model=embedding_model_name,
    ).data[0].embedding
    metadata = {}
    if author:
        metadata["author"] = author
    if tags:
        for tag in tags:
            metadata[tag] = True
    #
    results = v_table.ann_search(
        query_vector,
        n=n,
        metadata=metadata,
    )
    return [
        (result["body_blob"], result["metadata"]["author"])
        for result in results
    ]

### 検索機能のテスト

引用符のみを渡す場合：

In [14]:
find_quote_and_author("We struggle all our life for nothing", 3)

[('Life to the great majority is only a constant struggle for mere existence, with the certainty of losing it at last.',
  'schopenhauer'),
 ('We give up leisure in order that we may have leisure, just as we go to war in order that we may have peace.',
  'aristotle'),
 ('Perhaps the gods are kind to us, by making life more disagreeable as we grow older. In the end death seems less intolerable than the manifold burdens we carry',
  'freud')]

作成者に限定した検索：

In [15]:
find_quote_and_author("We struggle all our life for nothing", 2, author="nietzsche")

[('To live is to suffer, to survive is to find some meaning in the suffering.',
  'nietzsche'),
 ('What makes us heroic?--Confronting simultaneously our supreme suffering and our supreme hope.',
  'nietzsche')]

以前に引用符で保存したタグの中から検索を制限する：

In [16]:
find_quote_and_author("We struggle all our life for nothing", 2, tags=["politics"])

[('Mankind will never see an end of trouble until lovers of wisdom come to hold political power, or the holders of power become lovers of wisdom',
  'plato'),
 ('Everything the State says is a lie, and everything it has it has stolen.',
  'nietzsche')]

### 無関係な結果の除外

ベクトル類似度検索は一般的に、クエリに最も近いベクトルを返します。これは、より良い結果がない場合でも、やや無関係な結果を含む可能性があることを意味します。

この問題をコントロールするために、クエリと各結果の間の実際の「距離」を取得し、それに対してカットオフを設定することで、その閾値を超える結果を効果的に除外することができます。
この閾値を正しく調整することは簡単な問題ではありません。ここでは、その方法をお見せします。

これがどのように機能するかを理解するために、以下のクエリを試して、引用文と閾値の選択を変更して結果を比較してみてください：

_注記（数学的に興味のある方へ）：この「距離」は、ベクトル間のコサイン類似度、つまり2つのベクトルのノルムの積で割ったスカラー積と正確に同じです。そのため、これは-1から+1の範囲の数値で、-1は正確に反対方向を向いているベクトル、+1は同一方向を向いているベクトルを表します。他の場所（例：このデモの「CQL」対応版）では、この量を[0, 1]区間に収まるように再スケーリングしており、これは結果として得られる数値と適切な閾値がそれに応じて変換されることを意味します。_

In [17]:
quote = "Animals are our equals."
# quote = "Be good."
# quote = "This teapot is strange."

metric_threshold = 0.84

quote_vector = client.embeddings.create(
    input=[quote],
    model=embedding_model_name,
).data[0].embedding

results = list(v_table.metric_ann_search(
    quote_vector,
    n=8,
    metric="cos",
    metric_threshold=metric_threshold,
))

print(f"{len(results)} quotes within the threshold:")
for idx, result in enumerate(results):
    print(f"    {idx}. [distance={result['distance']:.3f}] \"{result['body_blob'][:70]}...\"")

3 quotes within the threshold:
    0. [distance=0.855] "The assumption that animals are without rights, and the illusion that ..."
    1. [distance=0.843] "Animals are in possession of themselves; their soul is in possession o..."
    2. [distance=0.841] "At his best, man is the noblest of all animals; separated from law and..."


## ユースケース2: **引用文ジェネレーター**

このタスクには、OpenAIの別のコンポーネント、つまりLLMが必要です。これは（Vector Storeへのクエリによって取得された入力に基づいて）見積もりを生成するためのものです。

また、generate-quote LLM完了タスクのために入力されるプロンプト用のテンプレートも必要です。

In [18]:
completion_model_name = "gpt-3.5-turbo"

generation_prompt_template = """"Generate a single short philosophical quote on the given topic,
similar in spirit and form to the provided actual example quotes.
Do not exceed 20-30 words in your quote.

REFERENCE TOPIC: "{topic}"

ACTUAL EXAMPLES:
{examples}
"""

検索と同様に、この機能は便利な関数にラップするのが最適です（内部的に検索を使用します）：

In [19]:
def generate_quote(topic, n=2, author=None, tags=None):
    quotes = find_quote_and_author(query_quote=topic, n=n, author=author, tags=tags)
    if quotes:
        prompt = generation_prompt_template.format(
            topic=topic,
            examples="\n".join(f"  - {quote[0]}" for quote in quotes),
        )
        # a little logging:
        print("** quotes found:")
        for q, a in quotes:
            print(f"**    - {q} ({a})")
        print("** end of logging")
        #
        response = client.chat.completions.create(
            model=completion_model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=320,
        )
        return response.choices[0].message.content.replace('"', '').strip()
    else:
        print("** no quotes found.")
        return None

_注意: 埋め込み計算の場合と同様に、Chat Completion APIのコードはv1.0以前のOpenAIでは若干異なります。_

#### 見積もり生成をテストする

テキストを渡すだけです（「引用」ですが、実際にはトピックを提案するだけでも構いません。そのベクトル埋め込みは、ベクトル空間内の適切な場所に配置されるためです）：

In [20]:
q_topic = generate_quote("politics and virtue")
print("\nA new generated quote:")
print(q_topic)

** quotes found:
**    - Happiness is the reward of virtue. (aristotle)
**    - Our moral virtues benefit mainly other people; intellectual virtues, on the other hand, benefit primarily ourselves; therefore the former make us universally popular, the latter unpopular. (schopenhauer)
** end of logging

A new generated quote:
Virtuous politics purifies society, while corrupt politics breeds chaos and decay.


単一の哲学者からのインスピレーションを使用する：

In [21]:
q_topic = generate_quote("animals", author="schopenhauer")
print("\nA new generated quote:")
print(q_topic)

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun! (schopenhauer)
**    - The assumption that animals are without rights, and the illusion that our treatment of them has no moral significance, is a positively outrageous example of Western crudity and barbarity. Universal compassion is the only guarantee of morality. (schopenhauer)
** end of logging

A new generated quote:
The true measure of humanity lies not in our dominion over animals, but in our ability to show com

## (オプション) **パーティショニング**

このクイックスタートを完了する前に、興味深いトピックを検討してみましょう。一般的に、タグと引用文は任意の関係を持つことができますが（例：引用文が複数のタグを持つ）、_著者_は実質的に正確なグループ化です（引用文の集合に対して「互いに素な分割」を定義します）：各引用文は正確に一人の著者を持ちます（少なくとも私たちにとって）。

さて、あなたのアプリケーションが通常（または常に）_単一の著者_に対してクエリを実行することが事前に分かっているとします。その場合、基盤となるデータベース構造を最大限に活用できます：引用文を**パーティション**にグループ化する（著者ごとに1つ）と、特定の著者のみに対するベクトルクエリはより少ないリソースを使用し、はるかに高速に結果を返します。

ここではCassandraストレージの内部に関わる詳細には踏み込みませんが、重要なメッセージは**クエリがグループ内で実行される場合、パフォーマンスを向上させるために適切にパーティション化することを検討する**ということです。

この選択が実際にどのように動作するかを見てみましょう。

まず、CassIOとは異なるテーブル抽象化が必要です：

In [22]:
from cassio.table import ClusteredMetadataVectorCassandraTable

In [23]:
v_table_partitioned = ClusteredMetadataVectorCassandraTable(table="philosophers_cassio_partitioned", vector_dimension=1536)

新しいテーブルに対してcompute-embeddings-and-insertステップを繰り返します。

先ほど見たものと比較して、重要な違いがあります。今度は引用の著者が、包括的な"metadata"辞書に追加される代わりに、挿入された行の_パーティションID_として保存されます。

ついでに、デモンストレーションとして、特定の著者によるすべての引用を_並行して_挿入します：CassIOでは、これは各引用に対して非同期の`put_async`メソッドを使用し、結果として得られる`Future`オブジェクトのリストを収集し、その後すべてに対して`result()`メソッドを呼び出して、すべてが実行されたことを確認することで行われます。Cassandra / Astra DBは、I/O操作において高度な並行性を十分にサポートしています。

_（注：以前に計算された埋め込みをキャッシュして、いくつかのAPIトークンを節約することもできましたが、ここではコードを検査しやすく保つことを優先しました。）_

In [24]:
BATCH_SIZE = 50

num_batches = ((len(philo_dataset) + BATCH_SIZE - 1) // BATCH_SIZE)

quotes_list = philo_dataset["quote"]
authors_list = philo_dataset["author"]
tags_list = philo_dataset["tags"]

print("Starting to store entries:")
for batch_i in range(num_batches):
    b_start = batch_i * BATCH_SIZE
    b_end = (batch_i + 1) * BATCH_SIZE
    # compute the embedding vectors for this batch
    b_emb_results = client.embeddings.create(
        input=quotes_list[b_start : b_end],
        model=embedding_model_name,
    )
    # prepare the rows for insertion
    futures = []
    print("B ", end="")
    for entry_idx, emb_result in zip(range(b_start, b_end), b_emb_results.data):
        if tags_list[entry_idx]:
            tags = {
                tag
                for tag in tags_list[entry_idx].split(";")
            }
        else:
            tags = set()
        author = authors_list[entry_idx]
        quote = quotes_list[entry_idx]
        futures.append(v_table_partitioned.put_async(
            partition_id=author,
            row_id=f"q_{author}_{entry_idx}",
            body_blob=quote,
            vector=emb_result.embedding,
            metadata={tag: True for tag in tags},
        ))
    #
    for future in futures:
        future.result()
    #
    print(f" done ({len(b_emb_results.data)})")

print("\nFinished storing entries.")

Starting to store entries:
B  done (50)
B  done (50)
B  done (50)
B  done (50)
B  done (50)
B  done (50)
B  done (50)
B  done (50)
B  done (50)

Finished storing entries.


この新しいテーブルでは、類似性検索も相応に変更されます（`ann_search`への引数に注意してください）：

In [25]:
def find_quote_and_author_p(query_quote, n, author=None, tags=None):
    query_vector = client.embeddings.create(
        input=[query_quote],
        model=embedding_model_name,
    ).data[0].embedding
    metadata = {}
    partition_id = None
    if author:
        partition_id = author
    if tags:
        for tag in tags:
            metadata[tag] = True
    #
    results = v_table_partitioned.ann_search(
        query_vector,
        n=n,
        partition_id=partition_id,
        metadata=metadata,
    )
    return [
        (result["body_blob"], result["partition_id"])
        for result in results
    ]

それで完了です：新しいテーブルは「汎用的な」類似検索を問題なくサポートしています...

In [26]:
find_quote_and_author_p("We struggle all our life for nothing", 3)

[('Life to the great majority is only a constant struggle for mere existence, with the certainty of losing it at last.',
  'schopenhauer'),
 ('We give up leisure in order that we may have leisure, just as we go to war in order that we may have peace.',
  'aristotle'),
 ('Perhaps the gods are kind to us, by making life more disagreeable as we grow older. In the end death seems less intolerable than the manifold burdens we carry',
  'freud')]

... しかし、作成者が指定された場合に、_大幅な_パフォーマンスの向上に気づくでしょう：

In [27]:
find_quote_and_author_p("We struggle all our life for nothing", 2, author="nietzsche")

[('To live is to suffer, to survive is to find some meaning in the suffering.',
  'nietzsche'),
 ('What makes us heroic?--Confronting simultaneously our supreme suffering and our supreme hope.',
  'nietzsche')]

まあ、現実的なサイズのデータセットがあれば、パフォーマンスの向上に_気づく_でしょう。このデモでは、数十個のエントリしかないため、目立った違いはありませんが、考え方は理解できるでしょう。

## 結論

おめでとうございます！OpenAIをベクトル埋め込みに使用し、Cassandra / Astra DBをCQLによるストレージに使用して、洗練された哲学的検索エンジンと引用生成器を構築する方法を学びました。

この例では[CassIO](https://cassio.org)を使用してVector Storeとのインターフェースを行いましたが、これが唯一の選択肢ではありません。他のオプションや人気のフレームワークとの統合については、[README](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases/cassandra_astradb)をご確認ください。

Astra DBのVector Search機能がML/GenAIアプリケーションの重要な要素となる方法について詳しく知りたい場合は、このトピックに関する[Astra DB](https://docs.datastax.com/en/astra/home/astra.html)のWebページをご覧ください。

## クリーンアップ

このデモで使用したすべてのリソースを削除したい場合は、このセルを実行してください（_警告: これによりテーブルとその中に挿入されたデータが削除されます！_）：

In [28]:
# we peek at CassIO's config to get a direct handle to the DB connection
session = cassio.config.resolve_session()
keyspace = cassio.config.resolve_keyspace()

session.execute(f"DROP TABLE IF EXISTS {keyspace}.philosophers_cassio;")
session.execute(f"DROP TABLE IF EXISTS {keyspace}.philosophers_cassio_partitioned;")